In [86]:

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from object_detection.utils import label_map_util
from object_detection.data_decoders import tf_example_decoder

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import os


from tensorflow.keras.layers import Conv2D, Reshape, Concatenate
from tensorflow.keras.models import Model


In [101]:

# Set up the data paths
train_tfrecord_dir = 'ThermalCamera_MLX90640.v3i.tfrecord/train/'
valid_tfrecord_dir = 'ThermalCamera_MLX90640.v3i.tfrecord/valid/'
train_label_map_path = os.path.join(train_tfrecord_dir, 'person_label_map.pbtxt')
valid_label_map_path = os.path.join(valid_tfrecord_dir, 'person_label_map.pbtxt')

# Load the label map
label_map = label_map_util.load_labelmap(train_label_map_path)
label_map_dict = label_map_util.get_label_map_dict(label_map)

# Get the list of TFRecord files
train_tfrecord_files = tf.io.gfile.glob(os.path.join(train_tfrecord_dir, '*.tfrecord'))
valid_tfrecord_files = tf.io.gfile.glob(os.path.join(valid_tfrecord_dir, '*.tfrecord'))

# Define the data loading function
def load_dataset(tfrecord_files, batch_size=32, shuffle_buffer_size=1000):
    """Loads a TFRecord dataset."""
    dataset = tf.data.TFRecordDataset(tfrecord_files)
    dataset = dataset.map(
        tf_example_decoder.TfExampleDecoder().decode, num_parallel_calls=tf.data.AUTOTUNE
    )
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

# Load the datasets
train_dataset = load_dataset(train_tfrecord_files)
valid_dataset = load_dataset(valid_tfrecord_files)


# for record in tf.data.TFRecordDataset(train_tfrecord_files):
#     decoded_record = tf_example_decoder.TfExampleDecoder().decode(record)
#     print(decoded_record)




In [102]:
import tensorflow as tf

# Get the first record file.
record_file = train_tfrecord_files[0]

# Create a dataset from the record file.
dataset = tf.data.TFRecordDataset(record_file)

# Decode the first example.
for raw_record in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    
    # Print the keys in this example.
    print(example.features.feature.keys())


KeysView({'image/encoded': bytes_list {
  value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\010\006\006\007\006\005\010\007\007\007\t\t\010\n\014\024\r\014\013\013\014\031\022\023\017\024\035\032\037\036\035\032\034\034 $.\' \",#\034\034(7),01444\037\'9=82<.342\377\333\000C\001\t\t\t\014\013\014\030\r\r\0302!\034!22222222222222222222222222222222222222222222222222\377\300\000\021\010\000\010\000\004\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\203\204\205\206\207\210\211\212\222\223\224\225\226\227\230\231\232\242\243\244\245\246\247\250\251\252\262\263\264\265\266\267\270\2

In [104]:
def create_ssd_mobilenetv2(num_classes):
    # PreprocessInputLayer 정의
    class PreprocessInputLayer(tf.keras.layers.Layer):
        def call(self, inputs):
            # 이미지를 3채널로 복제
            inputs = tf.concat([inputs, inputs, inputs], axis=-1)
            return inputs

    base_model = MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights=None)

    # Feature Extractor 부분: 여기서는 base_model의 중간 레이어들의 출력을 사용합니다.
    feature_layers = ['block_6_expand_relu', 'block_13_expand_relu', 'out_relu']

    features = [base_model.get_layer(layer).output for layer in feature_layers]
    
    # SSD Header 부분: 여기서는 각 feature map에 대해 Convolution과 Reshape을 적용합니다.
    headers = []
    
    for feature in features:
        header = Conv2D(6 * (num_classes + 4), kernel_size=3,
                        padding='same')(feature)
        header = Reshape((-1 ,num_classes + 4))(header)
        headers.append(header)

    # SSD Headers를 Concatenate합니다.
    predictions = Concatenate(axis=1)(headers)

    model = Model(inputs=base_model.input, outputs=predictions)
     
    return model

# 클래스 개수 설정 및 모델 생성 
num_classes = len(label_map.item) + 1 
model_ssd_mobilenetv2_gray_scale_input = create_ssd_mobilenetv2(num_classes=num_classes)

# Input shape 변경 및 Preprocessing Layer 추가 
inputs_gray_scale_32x32x1_input_shape=(32, 32, 1)
input_tensor_gray_scale_32x32x1=tf.keras.layers.Input(shape=inputs_gray_scale_32x32x1_input_shape)
preprocess_input_layer = PreprocessInputLayer()(input_tensor_gray_scale_32x32x1)
outputs_ssd_mobilenetv2=model_ssd_mobilenetv2_gray_scale_input(preprocess_input_layer)

model = Model(inputs=input_tensor_gray_scale_32x32x1, outputs=outputs_ssd_mobilenetv2)


# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

OperatorNotAllowedInGraphError: Exception encountered when calling layer "preprocess_input_layer_35" (type PreprocessInputLayer).

in user code:

    File "/var/folders/g2/t0zgyy0d1tl_58849t9hfg5c0000gn/T/ipykernel_15622/2037928495.py", line 10, in call  *
        height, width = tf.cast(tf.shape(inputs)[1:3], tf.float32)

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.


Call arguments received by layer "preprocess_input_layer_35" (type PreprocessInputLayer):
  • inputs=tf.Tensor(shape=(None, 32, 32, 1), dtype=float32)

In [ ]:
# 학습 설정
batch_size = 32
epochs = 10

# 학습
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
)


Epoch 1/10


ValueError: in user code:

    File "/Users/mac/anaconda3/envs/tf2/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/mac/anaconda3/envs/tf2/lib/python3.8/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/mac/anaconda3/envs/tf2/lib/python3.8/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/mac/anaconda3/envs/tf2/lib/python3.8/site-packages/keras/src/engine/training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "/Users/mac/anaconda3/envs/tf2/lib/python3.8/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/mac/anaconda3/envs/tf2/lib/python3.8/site-packages/keras/src/

In [ ]:
# 테스트 데이터셋 로드
test_tfrecord_dir = 'ThermalCamera_MLX90640.v1i.tfrecord/test/'
test_tfrecord_files = tf.io.gfile.glob(os.path.join(test_tfrecord_dir, '*.tfrecord'))
test_dataset = load_dataset(test_tfrecord_files)

# 모델 평가
eval_result = model.evaluate(test_dataset)

# 평가 결과 출력
print("Test Loss:", eval_result[0])
print("Test Accuracy:", eval_result[1])

# 모델 예측
predictions = model.predict(test_dataset)

# 예측 결과 확인
# (여기에서 예측 결과를 사용하여 바운딩 박스와 클래스 레이블을 추출하고 시각화할 수 있습니다)
